In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dense, Flatten
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime
import seaborn as sns
import matplotlib.pylab as plt
from matplotlib.font_manager import json_dump
import os
from sklearn.preprocessing import MinMaxScaler
import pickle

# tf.config.list_physical_devices("GPU")
# print(tf.__version__)
# tf.test.is_gpu_available(cuda_only=True, min_cuda_compute_capability=None)

data_dir = '../Dataset_sensor_floor/Dataset_Final/03082022_V2'
#data_dir = '../../Sensorfloors/summerschool2020/test/'
#os.path.join(dirname, train_data_dir)
#"path/to/training/data"

train = []
for file in os.listdir(data_dir):
    # Read a single file as pandas DataFrame
    print(file)
    df = pd.read_csv(os.path.join(data_dir, file))
    frames = []
    # Generate a single frame for each row
    for index, row in df.T.items():
        # Create dictionary with constant values (i.e. labels and frame number)
        frame = {col: row[col] for col in df.columns if col != 'data'}
        # Create pandas.DataFrame from Json String located in the 'data' column
        frame['data']  = pd.read_json(row['data'])
        frame['data']['timestamp'] = pd.to_datetime(frame['data']['timestamp'], unit='s')
        #print(pd.to_datetime(frame['data']['timestamp'], unit='ms'))
        frames.append(frame)
    train.append(frames)


2022-08-29 13:14:07.169375: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


dataset_0308022_moving_vertical_7-9_7-3_9-3_9-9.csv
dataset_0308022_moving_diag_21-10_5-3.csv
dataset_0308022_moving_vertical_17-9_17-3_20-3_20-9.csv
dataset_0308022_moving_vertical_10-9_10-3_13-3_13-9.csv
dataset_0308022_moving_vertical_7-9_6-3_4-3_4-9.csv
dataset_0308022_moving_vertical_18-9_18-3_15-3_15-9.csv
dataset_0308022_moving_edge.csv
dataset_0308022_moving_horizontal_21-6_4-6_21-9.csv
dataset_0308022_moving_horizontal_4-4_21-4.csv
dataset_0308022_moving_vertical_11-9_11-3_8-3_8-9.csv
dataset_0308022_moving_vertical_14-9_14-3_12-3_12-9.csv
dataset_0308022_moving_vertical_14-9_14-3_16-3_17-9.csv
dataset_0308022_moving_diag_5-11_21-3.csv


In [ ]:
train[12][900]

In [ ]:
timestamp_i = train[0][0]['data']['timestamp']
print(np.argmax(timestamp_i), timestamp_i[220])

**Define the range of min-max RSSI values**

In [ ]:
dataset_len = 0
rssi_max_frame = []
rssi_min_frame = []
rssi_max_threshold = []
rssi_min_threshold = []


for i in range(len(train)):
    for j in range(len(train[i])):
        rssi = train[i][j]['data']['r']
        rssi_min_frame.append(min(rssi))
        rssi_max_frame.append(max(rssi))
        #print("iteration: ", i, j, "val: ", min(rssi), min(rssi))
        
    rssi_min_threshold.append(min(rssi_min_frame))
    rssi_max_threshold.append(max(rssi_max_frame))
    print("train iteration: ", i, "val: ", min(rssi_min_frame),max(rssi_max_frame))
    rssi_min_frame.clear()
    rssi_max_frame.clear()

rssi_low_treshold = min(rssi_min_threshold)
rssi_upper_treshold = max(rssi_max_threshold)
print("train iteration: ", i, "val: ", rssi_low_treshold, rssi_upper_treshold)



**Generate Min Max scaler to normalize RSSI values**

In [2]:
rssi_values = []
for i in range(len(train)):
    for j in range(len(train[i])):
        for k in range(len(train[i][j]['data']['r'])):
            #rssi = train[i][j]['data']['r'][k]
            rssi_values.append(train[i][j]['data']['r'][k])
        #     rssi_values
        #
        #print(len(train[i][j]['data']['r']))
print(len(rssi_values))

9494400


In [7]:
scaler_rssi = MinMaxScaler(feature_range=(0, 1))
rssi_values_reshape = np.asarray(rssi_values)
rssi_values_reshape = rssi_values_reshape.reshape([-1,1])
scaler_rssi = scaler_rssi.fit(rssi_values_reshape)
norm_rssi = scaler_rssi.transform(rssi_values_reshape)

pickle.dump(scaler_rssi, open('output/scaler_rssi.pkl', 'wb'))

In [ ]:
norm_rssi